# Get List of Data Sources

In [ ]:
import requests, os
from requests.utils import requote_uri
urlString= 'https://api-dliservice-prototype-dli.d4science.org/v1/listDatasources'
r = requests.get(urlString)

#Create a URL encoded list of data sources
dataSources=[];
for ds in r.json():
    if len(ds.strip())>0:
        dataSources.append(requote_uri(ds))
        
        
#Print name of datasources
count=0
for ds in dataSources:
    print('{}. {}'.format(count,ds))
    count=count+1
    
print('We found {} data sources.'.format(count))

# Download files for a given datasource

In [ ]:
SelectedDataSources={4,8}

#Create local folder for the data sources
for i in SelectedDataSources:
    ds=dataSources[i]
    if not os.path.exists(ds):
        print ('Creating new folder: {}'.format(ds))
        os.makedirs(ds)
    else:
        print ('We found a local folder for: {}'.format(ds))

In [ ]:
#Download
downloadFiles=False
maximumPages=10001

if downloadFiles:
    for i in SelectedDataSources:
        ds=dataSources[i]
        page=0
        statusCode=200
        while (statusCode==200 and page < maximumPages):
            urlString= 'https://api-dliservice-prototype-dli.d4science.org/v1/linksFromDatasource?datasource={}&page={}'.format(ds,page)
            r = requests.get(urlString)
            fileName='{}.json'.format(1000000+page)
            myfile = open('./{}/{}'.format(ds,fileName), 'w')
            myfile.write(r.text)
            statusCode= r.status_code
            page = page + 1
            myfile.close
            if page%10==0:
                print('We have downloaded {} files for {}'.format(page,ds))
    print('Download is complete')
else:
    print('Download is disabled')

# Create CSV from JSON files

## Read JSON Files

In [ ]:
import glob, json, csv, pprint

nodes={}
relations=[]
for i in SelectedDataSources:
        ds=dataSources[i]
        path = ds
        dirs = os.listdir(path)
        path = '{}/*.json'.format(ds)
        for fname in glob.glob(path):
            print(fname)
            data = json.load(open(fname))

            for l in data:
                source_title=''
                target_title=''
                try:
                    source_title=l['source']['title']
                except Exception:
                    pass
                
                try:
                    target_title=l['source']['title']
                except Exception:
                    pass

                source_doi=''
                target_doi=''
                for i in l['source']['identifiers']:
                    if i['schema']=='dnetIdentifier':
                        source_id=i['identifier']
                    if i['schema']=='doi':
                        source_doi=i['identifier']

                for i in l['target']['identifiers']:
                    if i['schema']=='dnetIdentifier':
                        target_id=i['identifier']
                    if i['schema']=='doi':
                        target_doi=i['identifier']      

                label=''
                for p in l['linkProvider']:
                    label= label + p['name'] +';'
   
                relationship_type=l['relationship']['name']
    
                source_row=[source_id,source_title,source_doi,label]
                target_row=[target_id,target_title,target_doi,label]
                relation_row=[source_id,target_id,relationship_type]
                
                if source_id in nodes:
                    if source_doi!="":
                        nodes[source_id]=source_row
                else:
                    nodes[source_id]=source_row 

                if target_id in nodes:
                    if target_doi!="":
                        nodes[target_id]=target_row
                else:
                    nodes[target_id]=target_row
                   
                relations.append(relation_row)

print('Done!')

## Create nodes.csv

In [ ]:
nodeFile=open('nodes.csv','w')
nodeWriter = csv.writer(nodeFile, quoting=csv.QUOTE_ALL)
nodeWriter.writerow (['key:ID','title','doi','label'])

for row in nodes.values():
    nodeWriter.writerow (row)
    
nodeFile.close()
print('Done!')

## Create relations.csv

In [ ]:
nodeFile=open('relations.csv','w')
nodeWriter = csv.writer(nodeFile, quoting=csv.QUOTE_ALL)
nodeWriter.writerow ([':START_ID',':END_ID',':TYPE'])

for row in relations:
    nodeWriter.writerow(row)
    
nodeFile.close()
print('Done!')